# Experimentos

importamos las librerías necesarias para estos experimentos

In [121]:
import numpy as np
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors

## Funciones para leer imagenes, cargar voxeles de la imagen, etc.

In [42]:
def read_kImage(path):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(path)
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    return ct_scan

In [43]:
def extract_voxel_data(series_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames( series_path )
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image_3d = sitk.GetArrayFromImage(image)
    return image_3d

In [44]:
def extract_slice_data(img_3d, ijk):
    state = False
    k_string = ijk.split()[2]
    max_bandwith = img_3d.shape[0]
    possible_k = int (k_string)
    if (possible_k >= max_bandwith):
        possible_k = max_bandwith -1
        slice_index = possible_k
        state = True
        print("The ijk requested is {}, there was an error since the shapes of the image is {}".format(ijk,str(img_3d.shape)))
    else:
        slice_index = possible_k
        print("The ijk requested is {},OK since shapes of the image is {}".format(ijk,str(img_3d.shape)))
    return state, img_3d[slice_index,:,:]

## Funciones para generar tablas, editar tablas, operaciones entre tablas, etc.

In [45]:
def generate_training_tables():
    basedir = "../1/train/ProstateX-TrainingLesionInformationv2/ProstateX-TrainingLesionInformationv2"
    findingsfile = "{}/{}".format(basedir,"ProstateX-Findings-Train.csv")
    imagesfile = "{}/{}".format(basedir,"ProstateX-Images-Train.csv")
    kimagesfile = "{}/{}".format(basedir,"ProstateX-Images-KTrans-Train.csv")
    #Generamos las 3 tablas básicas a partir de los archivos.cvs
    findings_table = pd.read_csv(findingsfile)
    images_table = pd.read_csv(imagesfile)
    kimages_table = pd.read_csv(kimagesfile)
    #Operación de inner join entre tablas.
    join_kimages_table = pd.merge(left=findings_table, right= kimages_table, how ="inner", on=["ProxID","fid","pos"])
    join_images_table =  pd.merge(left=findings_table, right=  images_table, how ="inner", on=["ProxID","fid","pos"])
    #agregamos una columna para insertar la imagen
    join_images_table["MRI"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["MRI3D"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["State"] = pd.Series(False, index= join_images_table.index)
    join_kimages_table["Kimg"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["Kimg3D"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["State"] = pd.Series(False, index= join_kimages_table.index)
    return join_images_table, join_kimages_table

In [46]:
def set_MRI_value(index,image2d,image3d,state):
    join_images_table.at[index,"MRI"] = image2d
    join_images_table.at[index,"MRI3D"] = image3d
    join_images_table.at[index,"State"] = state

In [47]:
def set_kImg_value(index,image2d,image3d,state):
    join_kimages_table.at[index,"Kimg"] = image2d
    join_kimages_table.at[index,"Kimg3D"] = image3d
    join_kimages_table.at[index,"State"] = state

In [48]:
def insert_k_images():
    for row in range(len(join_kimages_table)):
        path = "../1/Train/ProstateXKtrains-train-fixed"
        patient_id = join_kimages_table.ProxID.iloc[row]
        ijk = join_kimages_table.ijk.iloc[row]
        path = "{}/{}".format(path,patient_id)
        path = "{}/{}-Ktrans.mhd".format(path,patient_id)
        kimage_3d = read_kImage(path)
        state, kimage_2d = extract_slice_data(kimage_3d,ijk)
        set_kImg_value(row,kimage_2d,kimage_3d,state)
        print("Se ha cargado la Kimagen2d y la kimagen3d #{} para el paciente {} ".format(row,patient_id))
        del path,patient_id,kimage_3d,state


In [49]:
def insert_MRI_images():
    for row in range (len(join_images_table)):
        patient_id = join_images_table.ProxID.iloc[row]
        series_name = join_images_table.DCMSerDescr.iloc[row]
        series_id = join_images_table.DCMSerNum.iloc[row]
        slice_ijk = join_images_table.ijk.iloc[row]
        path = "../1/Train/PROSTATEx/{}/".format(patient_id)#Selects the Patient ID
        path = "{}/{}".format(path,os.listdir(path)[0])#Selects the default folder inside the patient
        sequences = os.listdir(path) # Selects all the sequences
        sequence = [ sequence for sequence in sequences if sequence[0:len(str(series_id))] == str(series_id) ] [0]
        # Selects the only sequence that matches the condition.
        path = "{}/{}".format(path,sequence)
        image_3d = extract_voxel_data(path)
        state, image_2d = extract_slice_data(image_3d,slice_ijk)
        set_MRI_value(row,image_2d,image_3d,state)
        print("Se ha cargado la imagen MRI2D y MRI3D #{} para el paciente {} serie: {} ".format(row,patient_id,series_name))
        del path, patient_id, series_name, series_id,sequence,sequences, image_3d, image_2d

## Funciones para verificar la integridad de los datos.

In [177]:
def checkout_Kimgs_problems():
    k_problems_table = join_kimages_table[join_kimages_table["State"] == True] 
    print("There were {} problems".format(len(k_problems_table)))
    percent = 100*float(len(k_problems_table)/len(join_kimages_table))

    print("It was the  {0:.5f}% of the dataset".format(percent))
    display(k_problems_table[["ProxID","ijk","State", "zone"]])

In [51]:
def checkout_MRI_table_problems():
    MRI_problems_table = join_images_table[join_images_table["State"] == True] 
    print("There were {} problems".format(len(MRI_problems_table)))
    percent = 100*float(len(MRI_problems_table)/len(join_images_table))
    print("It was the  {0:.5f}% of the dataset".format(percent))
    display(MRI_problems_table[["ProxID","fid","ClinSig","State","Name","DCMSerDescr"]])
    
    patient_problems = MRI_problems_table.ProxID.unique()
    print("The following patients have almost 1 currupted image. {}".format(patient_problems))
    candidate_table = join_images_table
    for patient in patient_problems:
        candidate_table = candidate_table[candidate_table["ProxID"] != patient]
    print("This would be the table with none of the patients with atleast 1 problem.")
    display(candidate_table[["ProxID","fid","ClinSig","State","Name","DCMSerDescr"]])
    del MRI_problems_table, percent,  patient_problems, candidate_table, patient


## Funciones para generar un Dataframe de entrenamiento, datos de entrenamiento y labels de entrenamiento

In [52]:
def get_training_testing_tables(zone):
    #Base table with correct data and the zone requested.
    
    base_table = join_kimages_table[(join_kimages_table["State"] == False) & (join_kimages_table["zone"] == zone)]
    #Indexing zone for cancer and non-cancer rows.
    
    cancer_table_indexes = base_table[base_table["ClinSig"] == True].index.values
    non_cancer_table_indexes = base_table[base_table["ClinSig"] == False].index.values
    #Number of Samples for training and testing for cancer and non-cancer classes.
    
    training_cancer_samples = len(cancer_table_indexes)//2
    testing_cancer_samples = len(cancer_table_indexes) - training_cancer_samples
    training_non_cancer_samples = (len(non_cancer_table_indexes)//2)
    testing_non_cancer_samples = len(non_cancer_table_indexes) - training_non_cancer_samples
    #Training and testing table for cancer class
    
    training_cancer_table = join_kimages_table.iloc[cancer_table_indexes[:training_cancer_samples]]
    testing_cancer_table = join_kimages_table.iloc[cancer_table_indexes[training_cancer_samples:]]
    #Training and testing table for non-cancer class
    
    training_non_cancer_table = join_kimages_table.iloc[non_cancer_table_indexes[:training_non_cancer_samples]]
    testing_non_cancer_table = join_kimages_table.iloc[non_cancer_table_indexes[training_non_cancer_samples:]]
    #Integrity of the data

    #Cancer class
    print("There is a total of {} training cancer samples and a total of {} testing cancer samples".format(
        training_cancer_samples, testing_cancer_samples ))
    #Training

    print("These are the cancer table indexes for training {}".format(cancer_table_indexes[:training_cancer_samples]))
    display(training_cancer_table[["ProxID", "fid", "zone", "ClinSig", "State"]])
    #Testing

    print("These are the cancer table indexes for testing {}".format(cancer_table_indexes[training_cancer_samples:]))
    display(testing_cancer_table[["ProxID", "fid", "zone", "ClinSig", "State"]])
    #non_Cancer class

    print("There is a total of {} training non-cancer samples and a total of {} testing non-cancer samples".format(
        training_non_cancer_samples, testing_non_cancer_samples ))
    #Training

    print("These are the non_cancer table indexes for training {}".format(non_cancer_table_indexes[:training_non_cancer_samples]))
    display(training_non_cancer_table[["ProxID", "fid", "zone", "ClinSig", "State"]])
    #Testing
    print("These are the non_cancer table indexes for testing {}".format(non_cancer_table_indexes[training_non_cancer_samples:]))
    display(testing_non_cancer_table[["ProxID", "fid", "zone", "ClinSig", "State"]])

    #Joins training table and testing table.
    training_table = training_cancer_table.append(training_non_cancer_table)
    testing_table = testing_cancer_table.append(testing_non_cancer_table)
    training_table = training_table.sample(frac=1).reset_index(drop=True)
    testing_table = testing_table.sample(frac=1).reset_index(drop=True)
    print("This is the final training table")
    display(training_table[["ProxID", "fid", "zone", "ClinSig", "State"]])
    print("This is the final testing table")
    display(testing_table[["ProxID", "fid", "zone", "ClinSig", "State"]])

    return training_table,testing_table

In [53]:
def get_data_labels_Kimages(table):
    rows,cols = table.Kimg.iloc[0].shape
    data = np.ndarray(shape=(len(table), rows, cols),
                             dtype=np.float32)
    labels = np.ndarray(shape=(len(table),1),
                             dtype=np.float32)
    for index in range (len(table)):
        curr_rows,curr_cols = table.Kimg.iloc[index].shape
        if((curr_rows != rows or curr_cols != cols)):
            data[index, : , : ] = table.Kimg.iloc[index][:rows,:cols]
            labels[index] = table.ClinSig.iloc[index]
        else:
            data[index, : , : ] = table.Kimg.iloc[index]
            labels[index] = table.ClinSig.iloc[index]
    print("The shape of the data is: {}".format(data.shape) )
    data  = data.reshape((len(table),rows*cols))
    print("The shape of the data was reshaped to: {}".format(data.shape))
    labels = labels.reshape((len(table),))
    del table
    return data, labels

## Funciones para métricas estadísticas de los datos.

In [179]:
def get_TPR_TNR(testing_labels, predicted_labels):
    #Calculates the true positive rate and the true negative rate
    TP = 0
    TN = 0
    P = sum(testing_labels == True)
    N = sum(testing_labels == False)
    for i in range(len(testing_labels)):
        if (predicted_labels[i] == testing_labels[i]):
            if(testing_labels[i] == False):
                TN = TN + 1
            else:
                TP = TP + 1           
    TPR = TP/P
    TNR = TN/N
    return TPR, TNR , P , N

## Flujo principal de los Experimentos

In [54]:
join_images_table, join_kimages_table =  generate_training_tables()


In [178]:
insert_k_images()
print("==============================================================================================")
checkout_Kimgs_problems()

The ijk requested is 56 75 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #0 para el paciente ProstateX-0000 
The ijk requested is 51 61 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #1 para el paciente ProstateX-0001 
The ijk requested is 52 73 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #2 para el paciente ProstateX-0002 
The ijk requested is 68 72 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #3 para el paciente ProstateX-0002 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #4 para el paciente ProstateX-0003 
The ijk requested is 51 54 13,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #5 para el paciente ProstateX-0003 
The ijk requested is 74 56 6,OK since shapes of the image is (16, 128, 12

The ijk requested is 58 82 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #74 para el paciente ProstateX-0047 
The ijk requested is 77 70 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #75 para el paciente ProstateX-0048 
The ijk requested is 71 76 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #76 para el paciente ProstateX-0049 
The ijk requested is 73 64 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #77 para el paciente ProstateX-0050 
The ijk requested is 53 72 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #78 para el paciente ProstateX-0051 
The ijk requested is 71 48 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #79 para el paciente ProstateX-0052 
The ijk requested is 78 67 9,OK since shapes of the image is (16, 128,

The ijk requested is 58 56 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #141 para el paciente ProstateX-0095 
The ijk requested is 54 71 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #142 para el paciente ProstateX-0096 
The ijk requested is 57 60 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #143 para el paciente ProstateX-0097 
The ijk requested is 62 77 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #144 para el paciente ProstateX-0098 
The ijk requested is 77 79 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #145 para el paciente ProstateX-0099 
The ijk requested is 49 77 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #146 para el paciente ProstateX-0099 
The ijk requested is 54 51 5,OK since shapes of the image is (16

The ijk requested is 54 80 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #212 para el paciente ProstateX-0137 
The ijk requested is 60 83 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #213 para el paciente ProstateX-0138 
The ijk requested is 58 63 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #214 para el paciente ProstateX-0138 
The ijk requested is 55 83 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #215 para el paciente ProstateX-0138 
The ijk requested is 58 70 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #216 para el paciente ProstateX-0139 
The ijk requested is 52 76 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #217 para el paciente ProstateX-0139 
The ijk requested is 62 78 5,OK since shapes of the image is (1

The ijk requested is 53 68 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #282 para el paciente ProstateX-0173 
The ijk requested is 72 74 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #283 para el paciente ProstateX-0174 
The ijk requested is 72 62 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #284 para el paciente ProstateX-0175 
The ijk requested is 73 73 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #285 para el paciente ProstateX-0175 
The ijk requested is 69 62 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #286 para el paciente ProstateX-0176 
The ijk requested is 73 68 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d y la kimagen3d #287 para el paciente ProstateX-0176 
The ijk requested is 51 59 7,OK since shapes of the image is (

,ProxID,ijk,State,zone
34,ProstateX-0025,89 67 24,True,PZ
159,ProstateX-0105,63 78 17,True,SV
248,ProstateX-0154,57 72 16,True,SV


In [ ]:
insert_MRI_images()
print("==============================================================================================")
checkout_MRI_table_problems()

In [175]:
classifiers_table = pd.DataFrame(columns=["Classifier name","Prostate zone","Samples","Score", "TPR" , "P", "TNR", "N"])
zones = join_kimages_table[join_kimages_table["State"] == False].zone.unique()
for zone in zones:
    print("this is an experiment for the zone: {}".format(zone))
    training_table, testing_table = get_training_testing_tables(zone)
    print ("There is a total of {} patients with cancer at the training data".format(sum(training_table.ClinSig)))
    print ("There is a total of {} patients with cancer at the testing data".format(sum (testing_table.ClinSig)))
    training_data, training_labels = get_data_labels_Kimages(training_table)
    testing_data, testing_labels = get_data_labels_Kimages(testing_table)
    print("The data and the labels are ready for training and testing.") 
    print("=====================================================================================================")
    classifier1 = GaussianNB()
    classifier1.fit(training_data, training_labels)
    result = classifier1.score(testing_data,testing_labels)
    #Calculates the true positive rate and the true negative rate
    predicted_labels = classifier1.predict(testing_data)
    TPR, TNR , P , N = get_TPR_TNR(testing_labels, predicted_labels)
    print ("This was an experiment using GaussianNB classifier and only kimages of the {}.".format(zone)) 
    print("the number of samples in training were {} ".format(
    len(training_table)) + "and the number of samples in testing were {} ".format(len(testing_data)))
    print("the score of the accuracy in this experiment was {0:.5f}%".format(float(result)*100))
    print("This means that there was {}/{} correct predictions at testing.".format(int(result*len(testing_data)), len(testing_data) ))
    index = len(classifiers_table)
    classifiers_table.at[index,"Classifier name"] = "GaussianNB"
    classifiers_table.at[index,"Prostate zone"] = zone
    classifiers_table.at[index,"Score"] = result
    classifiers_table.at[index,"Samples"] = "{}/{}".format(int(result*len(testing_data)), len(testing_data) )
    classifiers_table.at[index,"TPR"] = TPR
    classifiers_table.at[index,"TNR"] = TNR
    classifiers_table.at[index,"P"] = P
    classifiers_table.at[index,"N"] = N
    print("=====================================================================================================")
    classifier2 = svm.SVC()
    classifier2.fit(training_data, training_labels)
    result = classifier2.score(testing_data,testing_labels)
    #Calculates the true positive rate and the true negative rate
    predicted_labels = classifier2.predict(testing_data)
    TPR, TNR , P , N = get_TPR_TNR(testing_labels, predicted_labels)
    print("=====================================================================================================")
    print ("This was an experiment using SVM-SVC classifier and only kimages of the {}.".format(zone)) 
    print("the number of samples in training were {} ".format(
    len(training_table)) + "and the number of samples in testing were {} ".format(len(testing_data)))
    print("the score of the accuracy in this experiment was {0:.5f}%".format(float(result)*100))
    print("This means that there was {}/{} correct predictions at testing.".format(int(result*len(testing_data)), len(testing_data) ))
    index = len(classifiers_table)
    classifiers_table.at[index,"Classifier name"] = "SVM-SVC"
    classifiers_table.at[index,"Prostate zone"] = zone
    classifiers_table.at[index,"Score"] = result
    classifiers_table.at[index,"Samples"] = "{}/{}".format(int(result*len(testing_data)), len(testing_data) )
    classifiers_table.at[index,"TPR"] = TPR
    classifiers_table.at[index,"TNR"] = TNR
    classifiers_table.at[index,"P"] = P
    classifiers_table.at[index,"N"] = N
    print("=====================================================================================================")
    print("=====================================================================================================")
    classifier3 = tree.DecisionTreeClassifier()
    classifier3.fit(training_data, training_labels)
    result = classifier3.score(testing_data,testing_labels)
    #Calculates the true positive rate and the true negative rate
    predicted_labels = classifier3.predict(testing_data)
    TPR, TNR , P , N = get_TPR_TNR(testing_labels, predicted_labels)
    print ("This was an experiment using Decision Tree classifier and only kimages of the {}.".format(zone)) 
    print("the number of samples in training were {} ".format(
    len(training_table)) + "and the number of samples in testing were {} ".format(len(testing_data)))
    print("the score of the accuracy in this experiment was {0:.5f}%".format(float(result)*100))
    print("This means that there was {}/{} correct predictions at testing.".format(int(result*len(testing_data)), len(testing_data) ))
    index = len(classifiers_table)
    classifiers_table.at[index,"Classifier name"] = "Decision Tree"
    classifiers_table.at[index,"Prostate zone"] = zone
    classifiers_table.at[index,"Score"] = result
    classifiers_table.at[index,"Samples"] = "{}/{}".format(int(result*len(testing_data)), len(testing_data) )
    classifiers_table.at[index,"TPR"] = TPR
    classifiers_table.at[index,"TNR"] = TNR
    classifiers_table.at[index,"P"] = P
    classifiers_table.at[index,"N"] = N
    print("=====================================================================================================")
    print("=====================================================================================================")
    n_estimators = 20
    classifier4 = RandomForestClassifier(n_estimators=n_estimators)
    classifier4.fit(training_data, training_labels)
    result = classifier4.score(testing_data,testing_labels)
    #Calculates the true positive rate and the true negative rate
    predicted_labels = classifier4.predict(testing_data)
    TPR, TNR , P , N = get_TPR_TNR(testing_labels, predicted_labels)
    print ("This was an experiment using Random Forest classifier and only kimages of the {}.".format(zone)) 
    print("the number of samples in training were {} ".format(
    len(training_table)) + "and the number of samples in testing were {} ".format(len(testing_data)))
    print("the score of the accuracy in this experiment was {0:.5f}%".format(float(result)*100))
    print("This means that there was {}/{} correct predictions at testing.".format(int(result*len(testing_data)), len(testing_data) ))
    index = len(classifiers_table)
    classifiers_table.at[index,"Classifier name"] = "Random Forest-{}".format(n_estimators)
    classifiers_table.at[index,"Prostate zone"] = zone
    classifiers_table.at[index,"Score"] = result
    classifiers_table.at[index,"Samples"] = "{}/{}".format(int(result*len(testing_data)), len(testing_data) )
    classifiers_table.at[index,"TPR"] = TPR
    classifiers_table.at[index,"TNR"] = TNR
    classifiers_table.at[index,"P"] = P
    classifiers_table.at[index,"N"] = N
    print("=====================================================================================================")
    print("=====================================================================================================")
    n_neighbors=20
    classifier5 = neighbors.KNeighborsClassifier(n_neighbors)
    classifier5.fit(training_data, training_labels)
    result = classifier5.score(testing_data,testing_labels)
    #Calculates the true positive rate and the true negative rate
    predicted_labels = classifier5.predict(testing_data)
    TPR, TNR , P , N = get_TPR_TNR(testing_labels, predicted_labels)
    print ("This was an experiment using KNeighbors classifier and only kimages of the {}.".format(zone)) 
    print("the number of samples in training were {} ".format(
    len(training_table)) + "and the number of samples in testing were {} ".format(len(testing_data)))
    print("the score of the accuracy in this experiment was {0:.5f}%".format(float(result)*100))
    print("This means that there was {}/{} correct predictions at testing.".format(int(result*len(testing_data)), len(testing_data) ))
    index = len(classifiers_table)
    classifiers_table.at[index,"Classifier name"] = "KNeighbors-{}".format(n_neighbors)
    classifiers_table.at[index,"Prostate zone"] = zone
    classifiers_table.at[index,"Score"] = result
    classifiers_table.at[index,"Samples"] = "{}/{}".format(int(result*len(testing_data)), len(testing_data) )
    classifiers_table.at[index,"TPR"] = TPR
    classifiers_table.at[index,"TNR"] = TNR
    classifiers_table.at[index,"P"] = P
    classifiers_table.at[index,"N"] = N
    print("=====================================================================================================")


this is an experiment for the zone: PZ
There is a total of 18 training cancer samples and a total of 18 testing cancer samples
These are the cancer table indexes for training [  0   2   7   8  25  45  49  64  67  72  97 112 140 145 153 176 180 184]


,ProxID,fid,zone,ClinSig,State
0,ProstateX-0000,1,PZ,True,False
2,ProstateX-0002,1,PZ,True,False
7,ProstateX-0005,0,PZ,True,False
8,ProstateX-0005,1,PZ,True,False
25,ProstateX-0019,1,PZ,True,False
45,ProstateX-0028,1,PZ,True,False
49,ProstateX-0031,1,PZ,True,False
64,ProstateX-0040,1,PZ,True,False
67,ProstateX-0041,1,PZ,True,False
72,ProstateX-0046,1,PZ,True,False


These are the cancer table indexes for testing [187 189 192 193 196 216 292 293 297 299 310 313 318 320 321 322 323 334]


,ProxID,fid,zone,ClinSig,State
187,ProstateX-0123,1,PZ,True,False
189,ProstateX-0125,1,PZ,True,False
192,ProstateX-0127,1,PZ,True,False
193,ProstateX-0128,1,PZ,True,False
196,ProstateX-0129,1,PZ,True,False
216,ProstateX-0139,1,PZ,True,False
292,ProstateX-0179,1,PZ,True,False
293,ProstateX-0179,2,PZ,True,False
297,ProstateX-0183,1,PZ,True,False
299,ProstateX-0184,2,PZ,True,False


There is a total of 79 training non-cancer samples and a total of 79 testing non-cancer samples
These are the non_cancer table indexes for training [  3  10  11  13  15  16  17  19  23  26  28  29  30  31  33  35  36  37
  38  41  42  44  46  47  48  50  51  52  54  57  59  61  62  63  65  66
  69  70  73  74  75  76  77  78  80  82  84  85  86  88  92  94  98  99
 100 101 108 109 111 116 118 121 122 128 137 138 144 146 149 151 154 156
 160 163 168 170 171 175 177]


...

This is the final training table


,ProxID,fid,zone,ClinSig,State
0,ProstateX-0068,3,TZ,False,False
1,ProstateX-0079,1,TZ,False,False
2,ProstateX-0062,1,TZ,False,False
3,ProstateX-0083,2,TZ,False,False
4,ProstateX-0026,1,TZ,True,False
5,ProstateX-0080,1,TZ,False,False
6,ProstateX-0105,1,TZ,True,False
7,ProstateX-0064,1,TZ,False,False
8,ProstateX-0086,2,TZ,False,False
9,ProstateX-0042,1,TZ,False,False


This is the final testing table


,ProxID,fid,zone,ClinSig,State
0,ProstateX-0108,2,TZ,False,False
1,ProstateX-0147,2,TZ,False,False
2,ProstateX-0093,1,TZ,False,False
3,ProstateX-0100,2,TZ,False,False
4,ProstateX-0135,1,TZ,False,False
5,ProstateX-0090,1,TZ,False,False
6,ProstateX-0161,3,TZ,False,False
7,ProstateX-0171,2,TZ,False,False
8,ProstateX-0107,1,TZ,False,False
9,ProstateX-0153,1,TZ,False,False


There is a total of 4 patients with cancer at the training data
There is a total of 5 patients with cancer at the testing data
The shape of the data is: (41, 128, 128)
The shape of the data was reshaped to: (41, 16384)
The shape of the data is: (42, 128, 128)
The shape of the data was reshaped to: (42, 16384)
The data and the labels are ready for training and testing.
This was an experiment using GaussianNB classifier and only kimages of the TZ.
the number of samples in training were 41 and the number of samples in testing were 42 
the score of the accuracy in this experiment was 88.09524%
This means that there was 37/42 correct predictions at testing.
This was an experiment using SVM-SVC classifier and only kimages of the TZ.
the number of samples in training were 41 and the number of samples in testing were 42 
the score of the accuracy in this experiment was 88.09524%
This means that there was 37/42 correct predictions at testing.
This was an experiment using Decision Tree classifie

In [180]:
display(classifiers_table)

,Classifier name,Prostate zone,Samples,Score,TPR,P,TNR,N
0,GaussianNB,PZ,75/97,0.773196,0.222222,18,0.898734,79
1,SVM-SVC,PZ,79/97,0.814433,0,18,1,79
2,Decision Tree,PZ,80/97,0.824742,0.222222,18,0.962025,79
3,Random Forest-20,PZ,79/97,0.814433,0,18,1,79
4,KNeighbors-20,PZ,79/97,0.814433,0,18,1,79
5,GaussianNB,AS,12/28,0.428571,0.625,16,0.166667,12
6,SVM-SVC,AS,15/28,0.535714,0.875,16,0.0833333,12
7,Decision Tree,AS,13/28,0.464286,0.625,16,0.25,12
8,Random Forest-20,AS,17/28,0.607143,0.8125,16,0.333333,12
9,KNeighbors-20,AS,16/28,0.571429,1,16,0,12
